Install TorchPysics library

In [ ]:
!pip install torchaudio==0.13.0
!pip install torchvision==0.14.0
!pip install torchphysics

In [3]:
import torch
import torchphysics as tp
import pytorch_lightning as pl

# PyTorch - Fully Connected Feed Forward Neural Network

In [3]:
class FCFFNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.call = torch.nn.Sequential(
            torch.nn.Linear(2, 5),
            torch.nn.Softplus(),
            torch.nn.Linear(5, 8),
            torch.nn.Tanh(),
            torch.nn.Linear(8, 3),
            torch.nn.Sigmoid(),
            torch.nn.Linear(3, 1)
        )

    def forward(self, x):
        return self.call(x)

Create an instance of FCFFNN and test basic properties of the mapping.

In [ ]:
simple_model = FCFFNN()

print(simple_model)

# access weights of matrix and bias-vector of first linear component
print(simple_model.call[0].weight)
print(simple_model.call[0].bias)

# generate a tensor
p = torch.Tensor([[0,0]])
print(p)
p.requires_grad = True
print(p)

# evaluate the network
test_eval = simple_model.forward(p)
print(test_eval)

# compute a (pathologic) gradient evaluation
test_grad =  torch.autograd.grad(test_eval, p, retain_graph=True)
print(test_grad)

# towards training/calibration the gradient w.r.t. the weights is relevant
test_grad_weight = torch.autograd.grad(test_eval,  [simple_model.call[0].weight, simple_model.call[0].bias])
print(test_grad_weight)

Workshop material including lecture notes, exercises (including the typical handwritten digits learning task using MNIST dataset; classification task bees vs ants data inclusive, ...) and sparse solutions available at https://github.com/Flo-Wo/DeepLearningWorkshop2023

# TorchPhysics
Based on exercises from a Workshop. Material including slides, exercises and solution proposals at - https://github.com/TomF98/torchphysics/tree/KoMSO-Workshop

Solve 2D Laplace Equation on Unit-Ball. That is

\begin{align*}
  \Delta u =& f \qquad \text{on }\Omega,\\
   u=& g \qquad \text{on } \partial\Omega,
\end{align*}

where for this example, we choose $$\Omega = \{x\in\mathbb{R} : \|x\|_2<1 \}$$ and $$f\colon\mathbb{R}^2→\mathbb{R};\; (x,y)\mapsto 4 \cos(x^2+y^2-1)-4(x^2+y^2)\sin(x^2+y^2-1).$$
The solution to this Poisson -Dirichlet Problem can be represented in closed form, and is given by the mapping
$$u\colon(x,y)\mapsto \sin(x^2+y^2-1).$$

In [67]:
X = tp.spaces.R2('x')
U = tp.spaces.R1('u')

omega = tp.domains.Circle(X, [0,0], 1)

Generate the model. That is a fully connected neural network (FCN).

In [68]:
model = tp.models.FCN(input_space=X,
                      output_space=U,
                      hidden=(50,50,50),
                      activations=torch.nn.Tanh(),
                      xavier_gains=1.0)

weight_pde = 1.0
weight_bound = 50.0
weight_data = 50.0

For training we want to minimize a loss functional of overall type

\begin{align*}
  \mathcal{L}(u) & = λ_1 \mathcal{L}_{\text{boundary}}(u) + λ_2\mathcal{L}_{\text{pde}}(u) + λ_3\mathcal{L}_{\text{data}}(u),
\end{align*}
where $\lambda_i\geq 0$. The individual loss components are given by
\begin{align*}
  \mathcal{L}_{\text{boundary}}(u) & =  \sum_{i} |u(x_i)-g(x_i)|^2, \qquad x_i \in \partial\Omega, \\
  \mathcal{L}_{\text{pde}}(u) &= \sum_i |\mathcal{F}(u)(x)|^2, \qquad x_i\in\Omega, \text{ and } \mathcal{F} \text{ the pde operator,}\\
  \mathcal{L}_{\text{data}}(u) & = \sum_i |u(x_i) - y_i|^2, \qquad x_i\in\Omega\cup\partial\Omega \text{ and } (x_i,y_i) \text{ is a labeled pair.}
\end{align*}


For the evaluation of $\mathcal{L}_{\text{boundary}}$ and $\mathcal{L}_{\text{pde}}$ we need points/mesh. This is done along so called *collocation points*. There is some freedom - theoretically a distribution according to an particle system can be derived - however here we compare different variations of the above loss functional.

#  PDE Loss Component - the physically driven part

In [69]:
inner_sampler = tp.samplers.RandomUniformSampler(omega, n_points=4000)
bound_sampler = tp.samplers.GridSampler(omega.boundary, n_points=250)

def boundary_residual(u, x):
    return u - 0.0

def pde_residual(u, x):
    laplace = tp.utils.laplacian(u, x)
    out_shape = laplace.shape
    norm_square = torch.reshape(torch.norm(x, dim=1)**2, out_shape)
    f = 4 * torch.cos(norm_square - 1) - 4 * norm_square * torch.sin(norm_square - 1)
    return laplace - f

boundary_cond = tp.conditions.PINNCondition(model, bound_sampler, boundary_residual, weight=weight_bound)
pde_cond = tp.conditions.PINNCondition(model, inner_sampler, pde_residual, weight=1.0)

# Data Loss Component

In [70]:
import numpy as np
import matplotlib.pyplot as plt

def exact_solution(x):
  return np.sin(np.linalg.norm(x)**2 - 1)

# labeled data domain
# entire domain
data_omega = omega
# restricted observations
# data_omega = tp.domains.Parallelogram(X, [-0.5,-0.5], [0.5,-0.5], [-0.5,0.5])

# generate points
data_sampler = tp.samplers.PlotSampler(data_omega, 200)
data_sampler.sample_points()

# generate labels; evaluate exact solution
coordinates_tensor = data_sampler.created_points.coordinates['x']
coordinates_np_array = coordinates_tensor.detach().numpy()
exact_solution_np_array = np.array([[exact_solution(p)] for p in coordinates_np_array])

# convert to torchphysics.Points
in_data_points = tp.spaces.Points(coordinates_tensor, X)
out_data_points = tp.spaces.Points(torch.Tensor(exact_solution_np_array), U)

Loss Data

In [71]:
data_loader = tp.utils.PointsDataLoader((in_data_points, out_data_points), batch_size=max(in_data_points.shape))

data_condition = tp.conditions.DataCondition(module=model,
                                             dataloader=data_loader,
                                             norm=2,
                                             use_full_dataset=True,
                                             weight=50.0)

Training

In [72]:
optim = tp.OptimizerSetting(optimizer_class=torch.optim.Adam, lr=0.0001)

# only physics
solver = tp.solver.Solver(train_conditions=[boundary_cond, pde_cond], optimizer_setting=optim)

# only data
# solver = tp.solver.Solver(train_conditions=[data_condition], optimizer_setting=optim)

# data-physics-mix
# solver = tp.solver.Solver(train_conditions=[boundary_cond, pde_cond, data_condition], optimizer_setting=optim)

trainer = pl.Trainer(gpus=1 if torch.cuda.is_available() else None,
                     max_steps=6000,
                     benchmark=True,
                     logger = False,
                     enable_checkpointing=False)

trainer.fit(solver)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name             | Type       | Params
------------------------------------------------
0 | train_conditions | ModuleList | 5.3 K 
1 | val_conditions   | ModuleList | 0     
------------------------------------------------
5.3 K     Trainable params
0         Non-trainable params
5.3 K     Total params
0.021     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


# Plotting

The relevant domains.

In [ ]:
# the entire domain
tp.utils.scatter(X, inner_sampler)

# the labeled data region
tp.utils.scatter(X, data_sampler)

Surface plot of approximation

In [ ]:
plot_sampler = tp.samplers.PlotSampler(plot_domain=omega,
                                       n_points=2000)
fig = tp.utils.plot(model, lambda u : u, plot_sampler)

# sanity check for x=(0,0)
pp = tp.problem.spaces.points.Points.from_coordinates({'x': torch.Tensor([[0,0]])})
print(model.forward(pp))

Levelset plotting

In [ ]:
x_axis = np.linspace(-1,1,100)
y_fix = 0.0

exact_values = np.array([exact_solution([x,y_fix]) for x in x_axis])
sample_points = tp.spaces.Points(torch.Tensor([[x, y_fix] for x in x_axis]), X)
predicted_values = torch.squeeze(model(sample_points)).detach().numpy()

plt.plot(x_axis, exact_values, color='r', linestyle='--', label='exact')
plt.plot(x_axis, predicted_values, color='b', label='prediction')
plt.title('Level-Set y = %2.2f' % y_fix)
plt.xlabel('x'), plt.ylabel('u(x,y)')
plt.legend(), plt.show()